# HW2 DATASCI W261: Machine Learning at Scale 

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  2
* **Date:**  5/23/16

**HW2.1.** Sort in Hadoop MapReduce.  **Given as input:** Records of the form (integer, “NA”), where integer is any integer, and “NA” is just the empty string.  **Output:** sorted key value pairs of the form (integer, “NA”) in decreasing order; what happens if you have multiple reducers? Do you need additional steps? Explain.

**Part 1** Write code to generate N  random records of the form (integer, “NA”). Let N = 10,000.

**Part 2** Write the python Hadoop streaming map-reduce job to perform this sort. Display the top 10 biggest numbers. Display the 10 smallest numbers

In [2]:
import random

N=10000
min_int = -100000
max_int = 100000
filename = "random_integers.txt"
with open(filename, 'w') as f:
    for i in range(N):
        f.write('<%d, "">\n' % (random.randint(min_int, max_int)))

In [9]:
# Create a directory in HDFS to store the data for this problem
#!/usr/local/hadoop/bin/hdfs dfs -mkdir /user
#!/usr/local/hadoop/bin/hdfs dfs -mkdir /user/hadoop

16/05/20 22:10:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
# Read the input file into HDFS
#!/usr/local/hadoop/bin/hdfs dfs -copyFromLocal random_integers.txt /user/hadoop

16/05/20 22:11:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
drwxr-xr-x   - hadoop supergroup          0 2016-05-20 22:10 /user/hadoop/outputHW2-1
-rw-r--r--   3 hadoop supergroup     123850 2016-05-20 22:09 /user/hadoop/random_integers.txt


In [22]:
# Check to make sure the file is there.
#!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop

16/05/20 22:33:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
-rw-r--r--   3 hadoop supergroup     123850 2016-05-20 22:09 /user/hadoop/random_integers.txt


In [17]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW2.1.

import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove the angle brackets
    line = line.strip('<>\n')
    # split the line into words
    words = line.split(',')
    # write the results to STDOUT (standard output);
    # what we output here will be the input for the
    # Reduce step, i.e. the input for reducer.py
    print('%s' % (words[0]))

Overwriting mapper.py


In [43]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW2.1.

import sys

N=10000
i=0
# input comes from STDIN
for line in sys.stdin:
    if (i>=0 and i<10) or (i>=(N-10) and i<N):
        print('<%s, "">' % (line.strip()))
    i += 1

Overwriting reducer.py


In [44]:
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-1/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options=-nr \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/random_integers.txt -output /user/hadoop/outputHW2-1

16/05/21 14:00:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/hadoop/outputHW2-1/*': No such file or directory
16/05/21 14:00:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 14:00:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar4921037629859414604/] [] /tmp/streamjob3968137248463036228.jar tmpDir=null
16/05/21 14:00:50 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/21 14:00:50 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/21 14:00:51 INFO mapred.FileInputFormat: Total input paths to process : 1
16/05/21 14:00:51 INFO mapreduce.JobSubmitter: number of splits:2
16/05/21 14:00:51 INFO Configuration.deprecati

In [45]:
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-1/part-00000

16/05/21 14:01:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-21 14:01 /user/hadoop/outputHW2-1/_SUCCESS
-rw-r--r--   3 hadoop supergroup        270 2016-05-21 14:01 /user/hadoop/outputHW2-1/part-00000
16/05/21 14:01:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<99999, "">	
<99995, "">	
<99982, "">	
<99974, "">	
<99969, "">	
<99942, "">	
<99929, "">	
<99892, "">	
<99887, "">	
<99882, "">	
<-99775, "">	
<-99842, "">	
<-99867, "">	
<-99889, "">	
<-99912, "">	
<-99939, "">	
<-99943, "">	
<-99961, "">	
<-99984, "">	
<-99994, "">	


In [46]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW2.2.

# Import sys library to access arguments passed in when the module is called
import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip('<>\n')
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print '%s\t%s' % (word, 1)

Overwriting mapper.py


In [47]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW2.2

from operator import itemgetter
# Import sys library to access arguments passed in when the module is called
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


In [48]:
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-1/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/random_integers.txt -output /user/hadoop/outputHW2-1

16/05/21 14:06:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 14:06:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-1/_SUCCESS
16/05/21 14:06:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-1/part-00000
16/05/21 14:06:21 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 14:06:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar7496711587575795057/] [] /tmp/streamjob8403600677380589246.jar tmpDir=null
16/05/21 14:06:23 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205

In [49]:
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-1/part-00000

16/05/21 14:06:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-21 14:06 /user/hadoop/outputHW2-1/_SUCCESS
-rw-r--r--   3 hadoop supergroup      91627 2016-05-21 14:06 /user/hadoop/outputHW2-1/part-00000
16/05/21 14:06:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
""	10000
-10013,	1
-10041,	1
-10059,	1
-10068,	2
-10090,	1
-10095,	1
-10116,	1
-10132,	1
-10164,	1
-10181,	1
-10227,	1
-10235,	1
-10246,	1
-10247,	1
-1026,	1
-10260,	1
-10273,	1
-10287,	1
-10306,	2
-10307,	1
-10333,	1
-10341,	1
-10348,	1
-10392,	1
-10405,	1
-10414,	1
-10421,	1
-10436,	1
-10454,	1
-1046,	1
-10465,	1
-10493,	1
-10497,	1
-10518,	1
-10545,	1
-10568,	1
-10585,	1
-10611,	1
-10660,	1
-10671,	1
-10686,	1
-10691,	1
-10710,	1
-10720,	1
-10766,	1
-10774,	1
-10781,	1
-10806,	1
-10822,	1
-

In [51]:
# Read the input file into HDFS
#!/usr/local/hadoop/bin/hdfs dfs -copyFromLocal enronemail_1h.txt /user/hadoop
# Check to make sure the file is there
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop

16/05/21 14:24:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
-rw-r--r--   3 hadoop supergroup     203981 2016-05-21 14:24 /user/hadoop/enronemail_1h.txt
drwxr-xr-x   - hadoop supergroup          0 2016-05-21 14:06 /user/hadoop/outputHW2-1
-rw-r--r--   3 hadoop supergroup     123850 2016-05-20 22:09 /user/hadoop/random_integers.txt


In [4]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: Mapper 1 code for HW2.3

# Import pring function from python 3
from __future__ import print_function
import sys
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the words in the 
# vocab that is being used for classification in each of the classes.
# Variable vocab:  stores the unique terms used in this file
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}
vocab = []

# input comes from STDIN (standard input)
for line in sys.stdin:
    # Split the line tabs and store the result in the 'items' list.
    items = line.split('\t')
    if len(items) == 1:
        continue
    # The 1st value of the line is the ID of the document
    ID = items[0]
    # The 2nd value of the line is the true classification of the document called: TRUTH
    TRUTH = int(items[1])
    # The 3rd value of the line is the subject.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    words_all = re.findall(WORD_RE, items[2].lower())
    # If the 4th value exists, it's the body of the line.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    if len(items) == 4:
        words_all = words_all + re.findall(WORD_RE, items[3].lower())
    # Update 'count_docs' by 1 for the class of this document
    count_docs[TRUTH] += 1
    # Update 'count_words' by the length of the 'words_all' list for the class of this document
    count_words[TRUTH] += len(words_all)
    # Create a variable to store the unique words from this line.
    vocab_line = []
    # Loop through each of the terms in the words of this line
    for term in words_all:
        # Update the vocab for this file
        if term not in vocab:
            vocab.append(term)
        # Update the vocab for this line
        if term not in vocab_line:
            vocab_line.append(term)
        # Initialize a list to store the count of this term
        if term not in count_vocab_terms:
            count_vocab_terms[term] = [0,0]
    # For each of the words in the vocabulary print out the 'term' and the '# of occurances
    # of the term' for reducer.py
    for term in vocab_line:
        # Store the number of occurances of the vocab word in the document
        count_term = words_all.count(term)
        # Update 'count_vocab_terms' for the class of this document
        count_vocab_terms[term][TRUTH] += count_term

# Print the count totals for each class that get passed to reducer.py
print('count,docs\t%d\t%d' % (count_docs[0], count_docs[1]))
print('count,words\t%d\t%d' % (count_words[0], count_words[1]))
for term in vocab:
    print('%s\t%d\t%d' % (term, count_vocab_terms[term][0], count_vocab_terms[term][1]))

Overwriting mapper.py


In [1]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: Reducer 1 code for HW2.3

# Import sys library to access arguments passed in when the module is called
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math

# Initialize counts.  All of the variables are lists storing 2 counts.  The 1st count
# in the list is the count for class 0 which is Ham and the 2nd count is the count
# for class 1 which is Spam.
# Variable count_docs:  stores the total number of documents for each class.
# Variable count_words:  stores the total number of words in the subject and body fields
# for each class.
# Variable count_vocab_terms:  stores the total number of occurances of the each of the 
# words in the vocab that is being used for classification in each of the classes.
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}

for line in sys.stdin:
    # Split the line by tabs
    elements = line.split("\t")
    # The 1st value in the line is the ID of the document
    label = elements[0]
    if label == 'count,docs':
        count_docs[0] += int(elements[1])
        count_docs[1] += int(elements[2])
    # Update the 'count_words' variable with the numbers from the file for each class
    elif label == 'count,words':
        count_words[0] += int(elements[1])
        count_words[1] += int(elements[2])
    # Update the 'count_vocab_terms' variable with the numbers from the file for each class
    else:
        term = label
        if term not in count_vocab_terms:
            count_vocab_terms[term] = [int(elements[1]), int(elements[2])]
        else:
            count_vocab_terms[term][0] += int(elements[1])
            count_vocab_terms[term][1] += int(elements[2])

# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Calculate the total number of documents by adding the total counts from each class.
total_docs = sum(count_docs)
# Initialize the 'prior' varialbe to store the prior probability for each class.
prior = [0, 0]
# Initialize the 'condprob' varialbe to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}
for term in count_vocab_terms:
    condprob[term] = [0, 0]

# Train the MultinomialNB classifer using the algorithm in the book: An Introduction to Information Retrieval
# By Christopher D. Manning, Prabhakar Raghavan & Hinrich Schutzepage page 260, Figure 13.2.
# For each of the classes:
# 1. Calculate the prior probability (prior) by dividing the total # of documents in that class by the total # of documents.
# 2. Calculate the conditional probability (condprob) for each term in the vocab by dividing the
# total # of that term in that class by the total number of terms in all documents in that class
# Use smoothing by adding 1.0 to the numerator and the denominator in the condprob equation.
# Create a variable to control if smoothing is used.
SMOOTHING = False
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
print('prior,prob\t%f\t%f' % (prior[0], prior[1]))
    
## SWAP THIS ONCE I SEE IT IS WORKING
for term in count_vocab_terms:
    for cls in classes:
        # If using smoothing, then add 1.0 to the numerator and the denominator in the condprob equation.
        if SMOOTHING:
            condprob[term][cls] = (count_vocab_terms[term][cls]+1.0) / (float(count_words[cls])+1.0)
        # If NOT using smoothing, then leave the numerator and the denominator alone.
        else:
            condprob[term][cls] = (count_vocab_terms[term][cls]) / (float(count_words[cls]))
    print('%s\t%f\t%f' % (term, condprob[term][0], condprob[term][1]))

Overwriting reducer.py


In [4]:
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-3a/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-3a
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/enronemail_1h.txt -output /user/hadoop/outputHW2-3a

16/05/21 22:48:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/hadoop/outputHW2-3a/*': No such file or directory
16/05/21 22:48:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rmdir: `/user/hadoop/outputHW2-3a': No such file or directory
16/05/21 22:48:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar4244948048980001265/] [] /tmp/streamjob9113684094145736531.jar tmpDir=null
16/05/21 22:48:59 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/21 22:48:59 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/21 22:49:00 INFO mapred.FileInputFormat: Total input paths to process : 1
16/05/21 22:49:00 INFO mapreduce.JobSubmitter: nu

In [5]:
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-3a
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-3a/part-00000

16/05/21 22:49:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-21 22:49 /user/hadoop/outputHW2-3a/_SUCCESS
-rw-r--r--   3 hadoop supergroup     140783 2016-05-21 22:49 /user/hadoop/outputHW2-3a/part-00000
16/05/21 22:49:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
prior,prob	0.560000	0.440000
yellow	0.000000	0.000054
barraged	0.000000	0.000054
prefix	0.000000	0.000054
railing	0.000000	0.000054
increase	0.000422	0.000326
granting	0.000000	0.000054
electricity	0.000070	0.000000
originality	0.000000	0.000054
homemakers	0.000000	0.000054
hormone	0.000000	0.000109
regional	0.000774	0.000000
screaming	0.000000	0.000054
four	0.000211	0.000272
internally	0.000141	0.000000
prize	0.000000	0.000054
customizable	0.000000	0.000054
wednesday	0.000422	0.000000
cyber

In [3]:
# Copy the model output from the 1st reducer to the local file system
!/usr/local/hadoop/bin/hdfs dfs -copyToLocal /user/hadoop/outputHW2-3a/part-00000 model.txt

16/05/21 23:29:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 23:29:10 WARN hdfs.DFSClient: DFSInputStream has been closed already


In [40]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: Mapper 2 code for HW2.3

# Import pring function from python 3
from __future__ import print_function
import sys
# Import the math library in order to use the 'log' method to take logorithms
import math
# Import re library to manipulate regular expressions
import re
# Define the regular expression used to designate what a 'word' is among a string of
# characters.
WORD_RE = re.compile(r"[\w']+")

# Create a list to store the 2 classes in this problem:  0 for Ham and 1 for Spam.
classes = [0, 1]
# Initialize the 'condprob' varialbe to store the conditional probilitity of each of the
# terms in the vocab each class.
condprob = {}

# Load the model from the model.txt file
with open("model.txt", 'r') as f:
    for line in f.readlines():
        elements = line.split('\t')
        if elements[0] == 'prior,prob':
            prior = [float(elements[1]), float(elements[2])]
        else:
            term = elements[0]
            condprob[term] = [float(elements[1]), float(elements[2])]

# Initialize a list to store the # of times a zero probability is processed for each class
count_docs_zero_prob = [0, 0]

for line in sys.stdin:
    # Read in the lines from the input file
    # Split the line tabs and store the result in the 'items' list.
    items = line.split('\t')
    if len(items) == 1:
        continue
    # The 1st value of the line is the ID of the document
    ID = items[0]
    # The 2nd value of the line is the true classification of the document called: TRUTH
    TRUTH = int(items[1])
    # The 3rd value of the line is the subject.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    words_all = re.findall(WORD_RE, items[2].lower())
    # If the 4th value exists, it's the body of the line.  Convert this value to lowercase and then
    # break it up into separate words using the WORD_RE regular expression
    if len(items) == 4:
        words_all = words_all + re.findall(WORD_RE, items[3].lower())
    # Make a prediction for each document based on the model
    score = [0, 0]
    for cls in classes:
        score[cls] = math.log(prior[cls])
        for term in words_all:
            # If any of the conditional probabilities for the document are 0, then set the score
            # to -100000000 because, in actuality the probabilities are being multiplied, so if one is 0
            # then the whole score will be 0.  If this happens, set the score to a very low number so that
            # this class will have a very low score and won't be selected and break out of the loop
            if condprob[term][cls] == 0.0:
                score[cls] = -1000000000
                count_docs_zero_prob[cls] += 1
                break
            else:
                score[cls] += math.log(condprob[term][cls])
    prediction = 1 if (score[1] > score[0]) else 0
    print("%s\t%d\t%d\t%f\t%f" % (ID, TRUTH, prediction, score[0], score[1]))

print('count,zeroprob\t%d\t%d' % (count_docs_zero_prob[0], count_docs_zero_prob[1]))

Overwriting mapper.py


In [34]:
%%writefile reducer.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: Reducer 2 code for HW2.3

import sys

# Initialize a list to store the # of times a zero probability is processed for each class
count_docs_zero_prob = [0, 0]
# Initialize variables to track the total incorrect predictions and the total predictions
count_total = 0
count_incorrect = 0
doc_probs = {}

for line in sys.stdin:
    items = line.split('\t')
    if items[0] == 'count,zeroprob':
        count_docs_zero_prob[0] += int(items[1])
        count_docs_zero_prob[1] += int(items[2])
    else:
        ID = items[0]
        TRUTH = int(items[1])
        prediction = int(items[2])
        if TRUTH != prediction:
            count_incorrect += 1
        count_total += 1
        doc_probs[ID] = [float(items[3]), float(items[4])]

print('Training Error: %f' % (count_incorrect / float(count_total)))
print('count,zeroprob\t%d\t%d' % (count_docs_zero_prob[0], count_docs_zero_prob[1]))
for ID, probs in doc_probs.iteritems():
    print('%s\t%f\t%f' % (ID, probs[0], probs[1]))

Overwriting reducer.py


In [41]:
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-3b/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-3b
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py,model.txt -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/enronemail_1h.txt -output /user/hadoop/outputHW2-3b

16/05/22 00:39:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/22 00:39:56 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-3b/_SUCCESS
16/05/22 00:39:56 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-3b/part-00000
16/05/22 00:39:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/22 00:39:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar3444388463207545335/] [] /tmp/streamjob428886361792397341.jar tmpDir=null
16/05/22 00:39:59 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.20

In [42]:
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-3b
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-3b/part-00000

16/05/22 00:40:57 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-22 00:40 /user/hadoop/outputHW2-3b/_SUCCESS
-rw-r--r--   3 hadoop supergroup       5470 2016-05-22 00:40 /user/hadoop/outputHW2-3b/part-00000
16/05/22 00:40:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Training Error: 0.000000	
count,zeroprob	44	56
0010.2003-12-18.GP	-1000000000.000000	-55.316830
0010.2001-06-28.SA_and_HP	-1000000000.000000	-3456.055866
0001.2000-01-17.beck	-3628.592339	-1000000000.000000
0018.1999-12-14.kaminski	-988.159991	-1000000000.000000
0005.1999-12-12.kaminski	-777.593930	-1000000000.000000
0011.2001-06-29.SA_and_HP	-1000000000.000000	-15962.696793
0008.2004-08-01.BG	-1000000000.000000	-5938.923501
0009.1999-12-14.farmer	-589.805873	-1000000000.000000
0017.2003-12-

In [15]:
with open("enronemail_1h.txt") as f:
    i = 0
    for line in f.readlines():
        count = line.count('\t')
        print('Line %d, %d' % (i, count))
        if count == 2:
            print(line)
        i += 1

Line 0, 3
Line 1, 3
Line 2, 3
Line 3, 2
0001.2000-06-06.lokay	0	 key dates and impact of upcoming sap implementation over the next few weeks, project apollo and beyond will conduct its final sap  implementation ) this implementation will impact approximately 12,000 new  users plus all existing system users. sap brings a new dynamic to enron,  enhancing the timely flow and sharing of specific project, human resources,  procurement, and financial information across business units and across  continents.  this final implementation will retire multiple, disparate systems and replace  them with a common, integrated system encompassing many processes including  payroll, timekeeping, benefits, project management, and numerous financial  processes.  employees will be empowered to update and/or view their personal information  via the intranet-based ehronline--a single front-end to sap's self service  functionality and enron's global information system (gis). among other  things, individuals w

In [11]:
!cat model.txt

prior,prob	0.560000	0.440000
yellow	0.000000	0.000054
barraged	0.000000	0.000054
prefix	0.000000	0.000054
railing	0.000000	0.000054
increase	0.000422	0.000326
granting	0.000000	0.000054
electricity	0.000070	0.000000
originality	0.000000	0.000054
homemakers	0.000000	0.000054
hormone	0.000000	0.000109
regional	0.000774	0.000000
screaming	0.000000	0.000054
four	0.000211	0.000272
internally	0.000141	0.000000
prize	0.000000	0.000054
customizable	0.000000	0.000054
wednesday	0.000422	0.000000
cyberopps	0.000000	0.000109
charter	0.000141	0.000000
tired	0.000000	0.000109
miller	0.000070	0.000000
bacon	0.000000	0.000054
tires	0.000000	0.000163
second	0.000070	0.000163
errors	0.000070	0.000272
thunder	0.000000	0.000054
daysor	0.000000	0.000054
bonuses	0.000000	0.000218
increasing	0.000281	0.000000
mailings	0.000000	0.000109
affiliates	0.000000	0.000326
47	0.000070	0.000054
china	0.000000	0.000109
borrowers	0.000000	0.000054
k	0.000070	0.000163
reports	0.000070	0.000000
oxymoron	0.000070	0.000000
